<a href="https://colab.research.google.com/github/cedro3/average_face/blob/main/average_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.セットアップ

In [ ]:
# --- e4e セットアップ ---
import os
os.chdir('/content')
CODE_DIR = 'encoder4editing'

!git clone https://github.com/cedro3/average_face.git $CODE_DIR
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force
os.chdir(f'./{CODE_DIR}')

from argparse import Namespace
import time
import os
import sys
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

sys.path.append(".")
sys.path.append("..")

from utils.common import tensor2im
from models.psp import pSp  # we use the pSp framework to load the e4e encoder.

%load_ext autoreload
%autoreload 2

# 学習済みパラメータのダウンロード
! pip install --upgrade gdown
import os
import gdown
os.makedirs('pretrained_models', exist_ok=True)
gdown.download('https://drive.google.com/u/1/uc?id=1Du_8FzOPKJhk6aJmiOBhAWVe3_6vAyET', 'pretrained_models/e4e_ffhq_encode.pt', quiet=False)

# ランドマークデータのダウンロード
! wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
! bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2

# モデルに学習済みパラメータをロード
model_path = 'pretrained_models/e4e_ffhq_encode.pt'  ####
ckpt = torch.load(model_path, map_location='cpu')
opts = ckpt['opts']
opts['checkpoint_path'] = model_path
opts= Namespace(**opts)
net = pSp(opts)
net.eval()
net.cuda()
print('Model successfully loaded!')

In [ ]:
# --- ライブラリーインポート＆関数定義 ---
%tensorflow_version 1.x
import numpy as np
import scipy.ndimage
import os
import PIL.Image
import sys
import bz2
from keras.utils import get_file
import dlib
import argparse
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import re
import projector
import pretrained_networks
from training import dataset
from training import misc
import matplotlib.pyplot as plt
from tqdm import trange


# -------------- フォルダー内画像表示 ---------------
def display_pic(folder):
    fig = plt.figure(figsize=(40, 40))
    files = os.listdir(folder)
    files.sort()
    for i, file in enumerate(files):
        img = Image.open(folder+'/'+file)    
        images = np.asarray(img)
        ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
        image_plt = np.array(images)
        ax.imshow(image_plt)
        fig.tight_layout()
        ax.set_xlabel(str(i+1), fontsize=30)               
    plt.show()
    plt.close()  

# -------------- ベクトルから画像を生成・保存 -------------
def vec2pic(vec_syn, dir):
  
    network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'  
    
    _G, _D, Gs = pretrained_networks.load_networks(network_pkl)
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

    Gs_syn_kwargs = dnnlib.EasyDict()
    Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_syn_kwargs.randomize_noise = True 
    Gs_syn_kwargs.truncation_psi = 0.5

    for i in range(len(vec_syn)):
        vec = vec_syn[i].reshape(1,18,512)
        image =  Gs.components.synthesis.run(vec, **Gs_syn_kwargs)        
        img = PIL.Image.fromarray(image[0])
        img.save(dir+str(i).zfill(3)+'.jpg') 

## 2.顔画像の切り出し
・imagesフォルダーの画像から所定の位置に合わせて顔部分を切り出し、alignフォルダーに保存します


In [ ]:
# 画像フォルダーの指定
path = './images/sample1'

In [ ]:
# --- 顔画像の切り出し ---
import os
import shutil
from tqdm import tqdm

if os.path.isdir('align'):
     shutil.rmtree('align')
os.makedirs('align', exist_ok=True)

def run_alignment(image_path):
  import dlib
  from utils.alignment import align_face
  predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
  aligned_image = align_face(filepath=image_path, predictor=predictor) 
  return aligned_image 

files = sorted(os.listdir(path))
for i, file in enumerate(tqdm(files)):
  if file=='.ipynb_checkpoints':
     continue
  input_image = run_alignment(path+'/'+file)
  input_image.resize((256,256))
  input_image.save('./align/'+file)

display_pic('align')

## 3.ベクトルの逆算
・alignフォルダーの画像からベクトルを逆算し、ベクトルをvecフォルダーに、そのベクトルから生成した画像をvec_picフォルダーへ保存します。

In [ ]:
# ------ ベクトルの逆算 ------

# フォルダーリセット
if os.path.isdir('vec_pic'):
     shutil.rmtree('vec_pic')
os.makedirs('vec_pic', exist_ok=True)

if os.path.isdir('vec'):
     shutil.rmtree('vec')
os.makedirs('vec', exist_ok=True)

img_transforms = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

path = './align'
files = sorted(os.listdir(path))
for i, file in enumerate(tqdm(files)):
  if file=='.ipynb_checkpoints':
     continue
  input_image = Image.open(path+'/'+file)
  transformed_image = img_transforms(input_image)
  with torch.no_grad():
     images, latents = net(transformed_image.unsqueeze(0).to('cuda').float(), randomize_noise=False, return_latents=True)
     result_image, latent = images[0], latents[0]
     tensor2im(result_image).save('./vec_pic/'+file) # vec_pic 保存
     torch.save(latents, './vec/'+file[:-4]+'.pt') # vec  保存

display_pic('vec_pic')

## 4.ベクトルの平均処理
・vecフォルダーのベクトルを平均し、ベクトルをvec_avgに、画像をvec_avg_picに保存します。

In [ ]:
# -----------  ベクトルの平均処理 ---------

# フォルダー・リセット
import os
import shutil
if os.path.isdir('vec_avg'):
     shutil.rmtree('vec_avg')
if os.path.isdir('vec_avg_pic'):
     shutil.rmtree('vec_avg_pic')
os.makedirs('vec_avg', exist_ok=True)
os.makedirs('vec_avg_pic', exist_ok=True)

# ベクトルの平均処理
import glob 
files = glob.glob('./vec/*.pt')
files.sort()

avg = 0
for i, file in enumerate(files):
     latent = torch.load(file)
     avg = (i*avg+latent)/(i+1)
     torch.save(avg, './vec_avg/'+str(i).zfill(3)+'.pt') # ベクトルを保存
     if i == 0:
        result = avg
     else:
        result = torch.cat((result, avg),0)
vec = result.to('cpu').detach().numpy().copy()

# ベクトルから画像を生成し保存
dir = 'vec_avg_pic/'
vec2pic(vec, dir)

# 保存した画像を表示
display_pic('vec_avg_pic')

## 5.平均画像の収束状況
・1〜N枚目の平均ベクトルと１枚目のベクトルのCOS類似度の関係を見る

In [ ]:
# --- 平均画像の収束状況確認 ---
value = []
for k in range(len(vec)):
    result = 0
    for i in range(18):
        cos = np.dot(vec[k,i],vec[0,i])/(np.linalg.norm(vec[k,i])*np.linalg.norm(vec[0,i]))
        result = result + cos/18
    value.append(result)

import matplotlib.pyplot as plt
plt.plot(value)
plt.ylabel('cos similarity')
plt.xlabel('N')

## 6.顔画像の多様性指数
・vecフォルダーのベクトルから平均ベクトルを計算し、各ベクトルと平均ベクトルとのCOS類似度から、顔画像の多様性指数を計算する。数値が大きいほど多様性が大きい。

In [ ]:
# -------- 多様性指数 ----------

# ベクトルの読み込み
import glob
files = glob.glob('vec/*.pt')
files.sort()

for i, file in enumerate(files):
    z = torch.load(file)
    if i == 0:
       vec = z
    else:
       vec = torch.cat((vec, z),0)

# 平均ベクトル計算（18, 512）
avg = 0
for i in range(len(vec)):
    avg = avg + vec[i]
avg = avg/len(vec)

# 各ベクトルと平均ベクトルのCOS類似度計算
var = 0
for i in range(len(vec)):
    tmp = torch.cosine_similarity(vec[i],avg)  # vec[i]と平均ベクトルとのCOS類似度
    tmp = torch.sum(tmp[2:8])/6  # 3〜8番目のみの平均をとる
    tmp = tmp.item()  # テンソルから数字を取り出す
    var = var + tmp
var = var/len(vec)  
var = 1 - var # 数値が大きいほど多様性が大きいにする
print('variance = ',var)

## 7.女性成分、男性成分

In [ ]:
# -------- 女性成分、男性成分 -------

# フォルダーリセット
import os
import shutil
if os.path.isdir('calc'):
     shutil.rmtree('calc')
os.makedirs('calc', exist_ok=True)
dir = 'calc/'

# 平均顔ベクトルの読み込み
x1 = torch.load('./sample/vector/1.pt')  # アジア系女性20人の平均顔
x2 = torch.load('./sample/vector/2.pt')  # アジア系女性10人＋欧米系女性10人の平均顔
x3 = torch.load('./sample/vector/3.pt')  # アジア系男性20人の平均顔
x4 = torch.load('./sample/vector/4.pt')  # アジア系男性10人＋欧米系男性10人の平均顔
x5 = torch.load('./sample/vector/5.pt')  # ミス・ジャパン35人の平均顔
x6 = torch.load('./sample/vector/6.pt')  # ミスター・ジャパン20人の平均顔
x7 = torch.load('./sample/vector/7.pt')  # ミス・インターナショナル30人の平均顔

# ベクトル演算
# ----------------------
z1 = x2 + (x2 - x4)*0.3  # 女性化＋30%
z2 = x2 + (x2 - x4)*0.5  # 女性化＋50%
z3 = x2 + (x2 - x4)*1.0  # 女性化＋100%
z = torch.cat((x2, z1, z2, z3), 0)  # （元画像, +30%, +50%, +100%）を表示
# ---------------------

z = z.to('cpu').detach().numpy().copy()
vec2pic(z, dir)
display_pic(dir)

# 8.美人度
・vecに保存されているベクトルとミス・インターナショナルの平均顔ベクトルとのCOS類似度を計算します

In [ ]:
# -------- 美人度 -----------

# ベクトルの読み込み
import glob
files = glob.glob('vec/*.pt')
files.sort()

for i, file in enumerate(files):
    z = torch.load(file)
    if i == 0:
       vec = z
    else:
       vec = torch.cat((vec, z),0)

# 基準ベクトルの読み込み
max = torch.load('./sample/vector/7.pt')  # ミス・インターナショナル30人の平均顔


# 美人度計算（各ベクトルと基準ベクトルとのCOS類似度）
for i in range(len(vec)):
    tmp = torch.cosine_similarity(vec[i],max[0])  # vec[i]と基準ベクトルとのCOS類似度
    tmp = torch.sum(tmp[2:8])/6  # 3〜8番目のみの平均をとる
    tmp = tmp.item()  # テンソルから数字を取り出す
    print(i+1, tmp)  # 各人のCOS類似度を表示